In [26]:
# Imports das Bibliotecas

# Bibliotecas necessárias
# Manipulação de dados
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
# Redes Neurais
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
# Plot
import matplotlib.pyplot as plt
%matplotlib inline
# Avaliação
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
df = pd.read_csv("updated_file.csv")

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1220 non-null   object
 1   Original_Text  1209 non-null   object
 2   Color          1220 non-null   object
 3   Label          1220 non-null   object
dtypes: object(4)
memory usage: 38.3+ KB


In [29]:
df = df.dropna(subset=["Original_Text"])
df = df.drop(columns=["Name"], axis=1)

# Limpeza dos Dados


In [30]:
df.value_counts()

Original_Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [31]:
nltk.download('stopwords')
stemmer = PorterStemmer() 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
def textPreprocessor(featureRecord):
    #Remove punctuations
    removePunctuation = [char for char in featureRecord if char not in string.punctuation]
    sentences =''.join(removePunctuation)

    #convert sentences to words
    words = sentences.split(" ")

    #normalize
    wordNormalized=[word.lower() for word in words]

    #remove stropwords
    finalWords=[word for word in wordNormalized if word not in stopwords.words("portuguese")]

    finalWords = [stemmer.stem(word) for word in finalWords]

    #joined = ' '.join(finalWords)

    return finalWords

In [33]:
!pip3 install spacy
!pip3 install unidecode

!python3 -m spacy download pt_core_news_sm


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe: No module named spacy


In [35]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from unidecode import unidecode

# Download Portuguese stopwords
stop_words = set(stopwords.words('portuguese'))

# Load the spaCy model for Portuguese
nlp = spacy.load("pt_core_news_sm")

def preprocess_text(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Retrieve the base form (lemma) of each word and remove punctuation
    lemmatized_words = [token.lemma_ for token in doc if token.text not in string.punctuation]

    # Remove stopwords and newline characters after lemmatization
    filtered_words = [word for word in lemmatized_words if word.lower() not in stop_words and word != '\n']

    # Remove accents from words
    cleaned_words = [unidecode(word) for word in filtered_words]

    # Reconstruct the text without stopwords, punctuation, accents, and newline characters
    preprocessed_text = ' '.join(cleaned_words)

    return preprocessed_text

# Example usage
text = "Estou correndo e pulando, mas com stopwords e \n que precisamos remover."
processed_text = preprocess_text(text)
print(processed_text)

correndo pular stopwords 
  precisar remover


In [36]:
df["Original_Text"] = df["Original_Text"].apply(preprocess_text)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Apply LabelEncoder to the 'Color' column
df['Color'] = label_encoder.fit_transform(df['Color'])

In [ ]:
y = df["Color"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer with stop words in Portuguese and using L2 normalization
Tfidf = TfidfVectorizer(norm='l2')

# Transform the text data in the "remainder__Original_Text" column of your DataFrame
X = Tfidf.fit_transform(df["Original_Text"]).toarray()

In [ ]:
from keras.initializers import glorot_uniform

def teste_model(flag): 

    n_neuronios_ocultos = 10
    iniciador_pesos = glorot_uniform(seed=42)
    
    tf.random.set_seed(42)
    model = tf.keras.Sequential()
    
    if flag == 1:
        model.add(tf.keras.Input(shape=X_train.shape[1:]))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu"))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu"))
        model.add(tf.keras.layers.Dense(3, activation='selu'))
    elif flag == 2:
        model.add(tf.keras.Input(shape=X_train.shape[1:]))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_initializer=iniciador_pesos))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_initializer=iniciador_pesos))
        model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
    elif flag == 3:
        model.add(tf.keras.Input(shape=X_train.shape[1:]))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(3, activation='softplus'))
    elif flag == 4:
        model.add(tf.keras.Input(shape=X_train.shape[1:]))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="selu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(3, activation='softmax'))
    elif flag == 5:
        model.add(tf.keras.Input(shape=X_train.shape[1:]))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="relu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(units=n_neuronios_ocultos, activation="relu", kernel_regularizer=tf.keras.regularizers.l2()))
        model.add(tf.keras.layers.Dense(3, activation='relu'))

    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model.compile(
        optimizer=optimizer,
        # Loss function to minimize
        loss="categorical_crossentropy",
        # loss=tf.keras.losses.CategoricalHinge(),
        # List of metrics to monitor
        metrics=["accuracy"]
    )
        
    model.build()
    model.summary()
    return model